In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np
import sklearn.linear_model
import sklearn.neighbors

In [2]:
datapath = os.path.join("datasets", "lifesat_gpd", "")

In [3]:
#Preapare oecd_bli data
oecd_bli = pd.read_csv(datapath + "oecd_bli_2015.csv", thousands=',')
oecd_bli = oecd_bli.query("INEQUALITY == 'TOT'")
oecd_bli = oecd_bli.pivot(index="Country", columns = "Indicator", values = "Value")
oecd_bli = oecd_bli["Life satisfaction"]



In [4]:
# Prepare gpd_per_capita data
gpd_per_capita = pd.read_csv(datapath + "gpd_per_capita.csv", thousands=',', delimiter='\t', encoding='latin1', na_values="n/a")
gpd_per_capita["GPD per Capita"] = gpd_per_capita["2015"]
gpd_per_capita = gpd_per_capita.set_index("Country")
gpd_per_capita = gpd_per_capita["GPD per Capita"]


In [5]:
# Merge
df = pd.merge(left=oecd_bli, right=gpd_per_capita, on="Country")
display(df.head())  

,Life satisfaction,GPD per Capita
Country,,
Australia,7.3,50961.865
Austria,6.9,43724.031
Belgium,6.9,40106.632
Brazil,7.0,8669.998
Canada,7.3,43331.961


In [6]:
fig = px.scatter(x="GPD per Capita", y="Life satisfaction", hover_data=["Country"], data_frame=df.reset_index(),)
fig.show()

In [7]:
np.c_?

Type:        CClass
String form: <numpy.lib.index_tricks.CClass object at 0x000001EAB3D42EB0>
Length:      0
File:        c:\users\simon\miniconda3\envs\mltraining\lib\site-packages\numpy\lib\index_tricks.py
Docstring:  
Translates slice objects to concatenation along the second axis.

This is short-hand for ``np.r_['-1,2,0', index expression]``, which is
useful because of its common occurrence. In particular, arrays will be
stacked along their last axis after being upgraded to at least 2-D with
1's post-pended to the shape (column vectors made out of 1-D arrays).

See Also
--------
column_stack : Stack 1-D arrays as columns into a 2-D array.
r_ : For more detailed documentation.

Examples
--------
>>> np.c_[np.array([1,2,3]), np.array([4,5,6])]
array([[1, 4],
       [2, 5],
       [3, 6]])
>>> np.c_[np.array([[1,2,3]]), 0, 0, np.array([[4,5,6]])]
array([[1, 2, 3, ..., 4, 5, 6]])


In [8]:
X = np.c_[df["GPD per Capita"]]
y = np.c_[df["Life satisfaction"]]

In [9]:
linear_model = sklearn.linear_model.LinearRegression()
knn_model = sklearn.neighbors.KNeighborsRegressor()

linear_model.fit(X,y)
knn_model.fit(X,y)

KNeighborsRegressor()

In [10]:
# Linear model prediction
t0, t1 = linear_model.intercept_[0], linear_model.coef_[0][0]
X_lin=np.linspace(0, 100000, 1000)
fig.add_scatter(x=X_lin, y= t0 + t1*X_lin, name="Linear model")

# Knn prediction
y_pred_knn = knn_model.predict(np.c_[X_lin])
fig.add_scatter(x=X_lin, y= y_pred_knn.flatten(), name="Knn model")

fig.show()